<a href="https://colab.research.google.com/github/bharat787/ForestDeptProj/blob/master/forestDeptResNet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import all libraries
main libraries include keras, numpy, cv2, and os

In [0]:
import numpy as np
import random
import cv2
import os
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
import scipy.misc
from sklearn.model_selection import train_test_split
from matplotlib.pyplot import imshow

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

Using TensorFlow backend.


# Identity Block
In this model we are experimenting with ResNet50 architecture.

The identity block is the standard block used in ResNets, and corresponds to the case where the input activation (say  a^[l] ) has the same dimension as the output activation (say  a^[l+2] ).

In [0]:
def identity_block(X, f, filters, stage, block):
    """
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    
    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    # Second component of main path (≈3 lines)
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)
    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    
    
    return X

# Convulational Block
The ResNet "convolutional block" is the second block type. You can use this type of block when the input and output dimensions don't match up. The difference with the identity block is that there is a CONV2D layer in the shortcut path.

* The CONV2D layer in the shortcut path is used to resize the input  xx  to a different dimension, so that the dimensions match up in the final addition needed to add the shortcut value back to the main path. 
* For example, to reduce the activation dimensions's height and width by a factor of 2, you can use a 1x1 convolution with a stride of 2.
* The CONV2D layer on the shortcut path does not use any non-linear activation function. Its main role is to just apply a (learned) linear function that reduces the dimension of the input, so that the dimensions match up for the later addition step

In [0]:
def convolutional_block(X, f, filters, stage, block, s = 2):
    """
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used
    
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path (≈3 lines)
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)
    # Third component of main path (≈2 lines)
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    ##### SHORTCUT PATH #### (≈2 lines)
    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X


# Defining ResNet50
Implementation of the popular ResNet50 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER


In [0]:
def ResNet50(input_shape = (64, 64, 3), classes = 2):
    """
    Implementation of the popular ResNet50 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    # Stage 3 (≈4 lines)
    X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4 (≈6 lines)
    X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5 (≈3 lines)
    X = X = convolutional_block(X, f=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')
    # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    X = AveragePooling2D(pool_size=(2, 2), padding='same')(X)
    

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [0]:
# initializing the model

model = ResNet50(input_shape = (64, 64, 3), classes = 2 )

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Since colab doesn't support cv2.imshow() we import a patch workaround cv2_imshow.

In [0]:
from google.colab.patches import cv2_imshow

# Load data and Preprocess
Find the path to your folder using the files menu on the right(if using colab) or use the path in your local system Preprocessing includes resizing to input size(here, 64x64), converting to numpy array, labelling the images, shuffling the whole dataset.

In [0]:
# code to preprocess and load data

# initialize the data and labels list
print("[INFO] loading images...")
data = []
labels = []

# grab the image paths and randomly shuffle them
imagePaths = '/content/drive/My Drive/forestdata/dataset/Mngroves' # address path
for filename in os.listdir(imagePaths):
    if filename.endswith("jpg"):                                   # change if other format
        # Your code comes here such as 
        fname = ('/content/drive/My Drive/forestdata/dataset/Mngroves/' + str(filename)) # total path by joining path + filename
        image = cv2.imread(fname)
        #cv2_imshow(image)
        image = cv2.resize(image, (64, 64))  # 64 x 64 is input to resnet
        image = img_to_array(image)
        data.append(image)
       
        # get the labels
        label = fname.split(os.path.sep)[-2]
        label = 1 if label == "Mngroves" else 0
        labels.append(label)
print('all done')   

In [0]:
imagePaths = '/content/drive/My Drive/forestdata/dataset/NtMngroves'
for filename in os.listdir(imagePaths):
    if filename.endswith("jpg"): 
        # Your code comes here such as 
        fname = ('/content/drive/My Drive/forestdata/dataset/NtMngroves/' + str(filename))
        print(fname)
        
        image = cv2.imread(fname)
        #cv2_imshow(image)
        image = cv2.resize(image, (64, 64))  # 64 x 64 is input to resnet
        image = img_to_array(image)
        data.append(image)
       
        # get the labels
        label = fname.split(os.path.sep)[-2]
        label = 1 if label == "Mngroves" else 0
        labels.append(label)
print('all done')

# Shuffling data
Done so as to prevent classifer to tune to one class over the other also both data and labels are shuffled in unison so that corresponding image and label remain the same.

In [0]:
rng_state = np.random.get_state()     # random state is kept the same so that
np.random.shuffle(data)               # both get same order whilst shuffling
np.random.set_state(rng_state)
np.random.shuffle(labels)

# Split Data and Create test and train sets
We will keep a proportion of images as trainset (here 75%) as train set rest as test set(25%). The random state helps to keep the test and train split always same to help debug while training the model.

In [0]:
(trainX, testX, trainY, testY) = train_test_split(data,
	labels, test_size=0.25, random_state=42)
print(type(trainY))
print(type(trainY))
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

# convert the labels from integers to vectors
trainY = to_categorical(trainY, num_classes=2)  # classification of 2 only
testY = to_categorical(testY, num_classes=2) 
print ("number of training examples = " + str(trainX.shape[0]))
print ("number of test examples = " + str(testX.shape[0]))
print ("X_train shape: " + str(trainX.shape))
print ("Y_train shape: " + str(trainY.shape))
print ("X_test shape: " + str(testX.shape))
print ("Y_test shape: " + str(testY.shape))

# Training and predicting the model


In [0]:
model.fit(trainX, trainY, epochs = 100, batch_size = 32)
'''
 too many epochs may cause your model to over-fit the training data. It means 
 that your model does not learn the data, it memorizes the data
 Generally batch size of 32 or 25 is good, with epochs = 100 unless you have 
 large dataset. in case of large dataset you can go with batch size of 10 with
  epochs b/w 50 to 100.
'''

In [0]:
preds = model.evaluate(testX, testY)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

Testing with your image.

In [0]:
img_path = '/content/drive/My Drive/forestdata/dataset/Mngroves/Mngroves1.jpg'
#img = image.load_img(img_path, target_size=(64, 64))
image = cv2.imread(img_path)
#cv2_imshow(image)
image = cv2.resize(image, (64, 64))
image = img_to_array(image)
x = image
x = np.expand_dims(x, axis=0)
x = x/255.0
print('Input image shape:', x.shape)
cv2_imshow(image)
print("class prediction vector [p(0), p(1) = ")
print(model.predict(x))

In [0]:
img_path = '/content/drive/My Drive/forestdata/dataset/Mngroves/Mngroves1.jpg'
image = cv2.imread(img_path)
image = cv2.resize(image, (1024, 1024))
cv2_imshow(image)

In [0]:
v